# Dart Report Crawler

## 0. imports

In [1]:
%load_ext jupyter_black

In [6]:
# import os
# import re
# import random
# import requests
# import time

# import dill
# import OpenDartReader
# import pandas as pd

# from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
# from tqdm.auto import tqdm

In [ ]:
# from dotenv import load_dotenv

# load_dotenv()

## 1. Class

In [6]:
# class DartCrawler:
#     def __init__(self):
#         self.dart = OpenDartReader(os.getenv("DART_API_KEY"))

#     def get_list(self, start_date: str, end_date: str, kind: str = "") -> pd.DataFrame:
#         self.start_date, self.end_date = start_date, end_date
#         start = datetime.strptime(start_date, "%Y-%m-%d")
#         end = datetime.strptime(end_date, "%Y-%m-%d")
#         dfs = []

#         while start < end:
#             next_month = start + timedelta(days=30)
#             if next_month > end:
#                 next_month = end

#             dfs.append(
#                 self.dart.list(
#                     start=start.strftime("%Y-%m-%d"),
#                     end=next_month.strftime("%Y-%m-%d"),
#                     kind=kind,
#                 )
#             )
#             start = next_month

#         self.list_df = pd.concat(dfs, ignore_index=True)
#         self.list_df = self.list_df.reset_index(drop=True)
#         return self.list_df

#     def get_document(self, list_df: pd.DataFrame, save_dir: str = "data") -> list[dict]:
#         self.data = []
#         for idx, row in tqdm(list_df.iterrows(), total=len(list_df)):
#             corp_code, corp_name = row["corp_code"], row["corp_name"]
#             rcept_no, report_nm = row["rcept_no"], row["report_nm"]

#             doc_df = self.dart.sub_docs(rcept_no)
#             document = {}
#             for idx, row in doc_df.iterrows():
#                 url = row["url"]
#                 title = row["title"]
#                 response = requests.get(url)
#                 soup = BeautifulSoup(response.text, "html.parser")
#                 text = soup.get_text(strip=False)
#                 text = re.sub(r"\n+", "\n", text)
#                 text = re.sub(r" {2,}", " ", text)

#                 document["title"] = title
#                 document["text"] = text

#             self.data.append(
#                 {
#                     "corp_code": corp_code,
#                     "corp_name": corp_name,
#                     "report_nm": report_nm,
#                     "document": document,
#                 }
#             )

#             time.sleep(random.uniform(0.3, 0.9))
#         if save_dir:
#             self._save_data(self.data, save_dir)

#         return self.data

#     def _save_data(self, data: list[dict], save_dir: str = "data") -> None:
#         if not os.path.exists(save_dir):
#             os.makedirs(save_dir)

#         save_path = os.path.join(
#             save_dir, f"dart_report_{self.start_date}_{self.end_date}.pkl"
#         )
#         with open(save_path, "wb") as f:
#             dill.dump(data, f)

In [9]:
# crawler = DartCrawler()

# list_df = crawler.get_list(start_date="2024-01-01", end_date="2024-01-02")
# data = crawler.get_document(list_df, save_dir="../data")

In [1]:
import sys

sys.path.append("..")

In [2]:
from src.dart_crawler import DartCrawler

In [3]:
crawler = DartCrawler()

list_df = crawler.get_list(start_date="2024-01-01", end_date="2024-04-30", kind='A')

In [4]:
list_df = list_df.sort_values(by='rcept_dt')

In [5]:
list_df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
34,01046203,스튜디오산타클로스,204630,K,[첨부정정]사업보고서 (2022.12),20240102000292,스튜디오산타클로스,20240102,연
32,01170962,GRT,900290,K,[기재정정]분기보고서 (2023.09),20240102000362,GRT,20240102,
33,01170962,GRT,900290,K,[기재정정]사업보고서 (2023.06),20240102000351,GRT,20240102,연
31,01188378,시스웍,269620,K,[기재정정]사업보고서 (2020.12),20240105000647,시스웍,20240105,연
30,01188378,시스웍,269620,K,[기재정정]사업보고서 (2021.12),20240105000651,시스웍,20240105,연
...,...,...,...,...,...,...,...,...,...
2970,00149646,기업은행,024110,Y,[기재정정]사업보고서 (2023.12),20240430000630,기업은행,20240430,연
2968,00168933,DB생명보험,,E,[기재정정]사업보고서 (2023.12),20240430000694,DB생명보험,20240430,연
2966,00909349,농협생명보험,,E,[기재정정]사업보고서 (2023.12),20240430000784,농협생명보험,20240430,연
2974,00148391,에이비엘생명보험,,E,[기재정정]사업보고서 (2023.12),20240430000559,에이비엘생명보험,20240430,연


In [6]:
data = crawler.get_document(list_df[:10], save_dir="../data")

  0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
data[-1]

{'corp_code': '01064388',
 'corp_name': '아론비행선박산업',
 'report_nm': '반기보고서 (2023.06)',
 'document': [{'title': '반 기 보 고 서',
   'text': '\n반 기 보 고 서\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 (제 16 기)\n사업연도\n2023년 01월 01일\n\xa0부터\n2023년 06월 30일\n\xa0까지\n금융위원회\n한국거래소 귀중\n2023 년 \xa0 06 \xa0월 \xa030 \xa0일\n제출대상법인 유형 :\n증권별 소유자수 500인 이상 외부감사대상법인\n면제사유발생 :\n해당사항 없음\n회 \xa0 \xa0 \xa0사 \xa0 \xa0 \xa0명 :\n아론비행선박산업 주식회사\n대 \xa0 표 \xa0 \xa0이 \xa0 사 :\n조현욱\n본 \xa0점 \xa0소 \xa0재 \xa0지 :\n경상남도 사천시 용현면 온정1길 63\n(전 \xa0화) 055-834-6556\n(홈페이지) http://www.aron.co.kr\n작 \xa0성 \xa0책 \xa0임 \xa0자 :\n(직 \xa0책) 상무이사 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0(성 \xa0명) 박준식\n(전 \xa0화) 055-834-6556\n'},
  {'title': '【 대표이사 등의 확인 】', 'text': '\n【 대표이사 등의 확인 】\n대표이사 등의 확인서\n'},
  {'title': 'I. 회사의 개요',
   'text': "\nI. 회사의 개요\n1. 회사의 개요\n1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이 사업보고서, 분기ㆍ반기보고서를 제출하는 경우에 한함)연결대상 종속회사 현황(요약)\n(단위 : 사)\n구분\n연결대상회사수\n주요종속회사수\n기초\n증가\n감소\n기말\n상장\n-\n-\n